In [26]:
import networkx as nx
import numpy as np
from qiskit import QuantumCircuit
# from qiskit.circuit.library import Permutation
from sympy.combinatorics import Permutation
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SabreSwap, SabreLayout, ApplyLayout, FullAncillaAllocation, EnlargeWithAncilla

In [27]:
n = 4
circ = QuantumCircuit(n)
for i in range(n):
    for j in range(i):
        circ.cx(i, j)

cm = CouplingMap(
    [(i, i+1) for i in range(n)]
)

In [28]:
print(circ)


     ┌───┐┌───┐     ┌───┐          
q_0: ┤ X ├┤ X ├─────┤ X ├──────────
     └─┬─┘└─┬─┘┌───┐└─┬─┘┌───┐     
q_1: ──■────┼──┤ X ├──┼──┤ X ├─────
            │  └─┬─┘  │  └─┬─┘┌───┐
q_2: ───────■────■────┼────┼──┤ X ├
                      │    │  └─┬─┘
q_3: ─────────────────■────■────■──
                                   


In [29]:
sabre_layout = SabreLayout(cm, seed=42)
pm = PassManager([sabre_layout])


In [30]:
mapped_circ = pm.run(circ)

In [31]:
print(mapped_circ)


                                                 
ancilla_0 -> 0 ──────────────────────────────────
                                                 
      q_3 -> 1 ───────────────■───X──────────────
                            ┌─┴─┐ │         ┌───┐
      q_2 -> 2 ───────■───X─┤ X ├─X──X──────┤ X ├
               ┌───┐┌─┴─┐ │ └───┘    │      └─┬─┘
      q_0 -> 3 ┤ X ├┤ X ├─X───■──────X───■────■──
               └─┬─┘└───┘   ┌─┴─┐      ┌─┴─┐     
      q_1 -> 4 ──■──────────┤ X ├──────┤ X ├─────
                            └───┘      └───┘     


In [32]:
mapped_circ.layout.initial_layout

Layout({
3: Qubit(QuantumRegister(4, 'q'), 0),
4: Qubit(QuantumRegister(4, 'q'), 1),
2: Qubit(QuantumRegister(4, 'q'), 2),
1: Qubit(QuantumRegister(4, 'q'), 3),
0: Qubit(QuantumRegister(1, 'ancilla'), 0)
})

In [33]:
pattern = mapped_circ.layout.routing_permutation()
print(pattern)

[0, 3, 2, 1, 4]


In [34]:
print(Permutation(pattern).cyclic_form)

[[1, 3]]


In [35]:
# P = Permutation(len(pattern), pattern).decompose()

In [36]:
# P = Permutation(len(pattern), pattern).decompose()
# print(P)

In [37]:
CG = np.array(
    [
        [0, 1, 0, 0],
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 0, 1, 0],
    ]
)
G = nx.from_numpy_array(CG)


In [38]:
# mapped_circ.barrier()

# for s in P.data:
#     q0 = s.qubits[0]._index
#     q1 = s.qubits[1]._index
#     path = nx.dijkstra_path(G, q0, q1)
#     for i in range(len(path)-2):
#         mapped_circ.swap(path[i], path[i+1])
#     mapped_circ.swap(path[-2], path[-1])
#     for i in reversed(range(len(path)-2)):
#         mapped_circ.swap(path[i], path[i+1])
    

In [39]:
print(mapped_circ)

                                                 
ancilla_0 -> 0 ──────────────────────────────────
                                                 
      q_3 -> 1 ───────────────■───X──────────────
                            ┌─┴─┐ │         ┌───┐
      q_2 -> 2 ───────■───X─┤ X ├─X──X──────┤ X ├
               ┌───┐┌─┴─┐ │ └───┘    │      └─┬─┘
      q_0 -> 3 ┤ X ├┤ X ├─X───■──────X───■────■──
               └─┬─┘└───┘   ┌─┴─┐      ┌─┴─┐     
      q_1 -> 4 ──■──────────┤ X ├──────┤ X ├─────
                            └───┘      └───┘     


In [40]:
map_source = [3, 2]
map_target = [0, 2]

In [41]:
DataNum = 3
Swaps= []
while set(map_target).difference(map_source):
    q = set(map_target).difference(map_source).pop()
    idx = map_target.index(q)
    Swaps.append((map_source[idx], map_target[idx]))
    map_source[idx] = map_target[idx]
print(Swaps)

[(3, 0)]


In [42]:
print(map_source, map_target)

[0, 2] [0, 2]


In [43]:
if map_source != map_target:
    pattern = []
    for idx in range(DataNum):
        if map_source[idx] == map_target[idx]:
            pattern.append(idx)
        else:
            pattern.append(map_target.index(map_source[idx]))
    for idx0, idx1 in Permutation(pattern).cyclic_form:
        Swaps.append((map_source[idx0], map_source[idx1]))
print(Swaps)

[(3, 0)]


In [44]:
Swap_layer = []
map_source = [3, 2]

for q0, q1 in Swaps:
    path = nx.dijkstra_path(G, q0, q1)
    for i, j in zip(path[:-1], path[1:]):
        if i in map_source or j in map_source:
            map_source = [i if q == j else (j if q == i else q) for q in map_source]
            Swap_layer.append([i, j])

    for i, j in zip(reversed(path[:-2]), reversed(path[1:-1])):
        if i in map_source or j in map_source:
            map_source = [i if q == j else (j if q == i else q) for q in map_source]
            Swap_layer.append([i, j])

Swap_layer
assert map_source == map_target

In [45]:
print()

In [46]:
QC.to_instruction()

NameError: name 'QC' is not defined

In [ ]:
print(QC.decompose([QC.to_instruction().name]))

                     
 q_0: ─────────────X─
                   │ 
 q_1: ─────────────┼─
                   │ 
 q_2: ──────────X──X─
                │    
 q_3: ──────────┼────
                │    
 q_4: ──────────┼──X─
                │  │ 
 q_5: ───────X──X──┼─
             │     │ 
 q_6: ───────┼─────┼─
             │     │ 
 q_7: ───────┼─────┼─
             │     │ 
 q_8: ───────┼─────┼─
             │     │ 
 q_9: ───────┼─────┼─
             │     │ 
q_10: ───────┼─────┼─
             │     │ 
q_11: ─X─────┼─────┼─
       │     │     │ 
q_12: ─┼─────┼─────┼─
       │     │     │ 
q_13: ─┼──X──┼─────┼─
       │  │  │     │ 
q_14: ─X──┼──┼─────┼─
          │  │     │ 
q_15: ────X──X─────X─
                     
